In [1]:
import numpy as np
import xarray as xr 
import glob
import dask.array as da
from tqdm import tqdm
import pandas as pd
from dask.distributed import Client
from scipy.interpolate import PchipInterpolator
from scipy import stats
import matplotlib.pyplot as plt

In [8]:
print('numpy version', np.__version__)
print('xarray version', xr.__version__)
import dask
print('dask version', dask.__version__)
print('pandas version', pd.__version__)
import scipy
print('scipy version', scipy.__version__)
import matplotlib
print('matplotlib version', matplotlib.__version__)

numpy version 1.24.3
xarray version 2023.4.2
dask version 2023.4.1
pandas version 1.5.3
scipy version 1.10.1
matplotlib version 3.8.4


In [ ]:
from dask.distributed import Client
client = Client(
    scheduler_file='/*your_path/scheduler.json')
client


<Client: 'tcp://203.247.189.224:45950' processes=16 threads=288, memory=298.02 GiB>

In [ ]:
# prepare the basic information for the CLM5 grids
lake_ixy = xr.open_dataarray('lake_ixy.nc')
lake_jxy = xr.open_dataarray('lake_jxy.nc')
zlake = xr.open_dataarray('ZLAKE.nc')
zlake_lunit = xr.open_dataarray('ZLAKE_LUNIT.nc')

In [ ]:
'''
monthly lake temperture data (TLAKE) can be access at https://www.earthsystemgrid.org/ (CESM2 Large Ensemble)
and in order to save computation resources, we convert the monthly TLAKE from a 2D (latxlon) grid to sequential (1D) grid and drop the grids not having lakes by following method:
tlake_lunit = tlake[:,:,lake_ixy, lake_jxy]
each monthly file are in the following structures (landunit is the sequential grid that have already dropped grids not having lakes), "1231.019" is the member number in the large ensemble
#------------------------------------------------
ncdump -h TLAKE_MON12_LUNIT_1231.019.nc                                    
netcdf TLAKE_MON12_LUNIT_1231.019 {
dimensions:
        landunit = 4622 ;
        year = 251 ;
        levlak = 10 ;
variables:
        float lon(landunit) ;
                lon:_FillValue = 1.e+36f ;
                lon:long_name = "coordinate longitude" ;
                lon:units = "degrees_east" ;
                lon:missing_value = 1.e+36f ;
        float lat(landunit) ;
                lat:_FillValue = 1.e+36f ;
                lat:long_name = "coordinate latitude" ;
                lat:units = "degrees_north" ;
                lat:missing_value = 1.e+36f ;
        int64 year(year) ;
        string ensemble ;
        int64 landunit(landunit) ;
        float TLAKE(year, levlak, landunit) ;
                TLAKE:_FillValue = 1.e+36f ;
                TLAKE:long_name = "lake temperature" ;
                TLAKE:units = "K" ;
                TLAKE:cell_methods = "time: mean" ;
                TLAKE:coordinates = "lon lat" ;
                TLAKE:missing_value = 1.e+36f ;

// global attributes:
                :coordinates = "ensemble" ;
}
#------------------------------------------------
'''

# calculate TOE of seasonal mean

In [ ]:
years = np.arange(1850, 2101, dtype='float32')


def toe(x):
    if x.any():
        return years[x][-1] + 1
    else:
        return np.nan


np.set_printoptions(1, suppress=True)
depths = np.concatenate(
    (np.arange(0.5, 15.1, 0.5),
     np.arange(17, 31, 2),
     np.arange(31, 100, 8),
     np.arange(103, 700, 64)), dtype='float'
)


In [ ]:
ens_numbers = [a[-11:-3]
               for a in sorted(glob.glob('*your_path/TLAKE_MON7_LUNIT*nc'))]
DIC = '*your_path/TLAKE_MON/Single_Ens/'
files = [[DIC + 'TLAKE_MON' + str(b) + '_LUNIT_' + a +
          '.nc' for b in np.arange(1, 13)] for a in ens_numbers]

tlake_mon_lunit = xr.open_mfdataset(
    files,
    combine='nested',
    #chunks={'ensemble':100},
    concat_dim=[[*ens_numbers], 'season'],
    parallel=True
).drop(('ensemble', 'lat', 'lon')).rename({'concat_dim': 'ensemble'}).TLAKE\
    .transpose('levlak', ...).assign_coords(levlak=zlake_lunit.levlak)\
    .chunk({'ensemble': 100, 'landunit': 1})

seasons = ['JFM', 'JFM', 'JFM', 'AMJ', 'AMJ',
           'AMJ', 'JAS', 'JAS', 'JAS', 'OND', 'OND', 'OND']
tlake_mon_lunit = tlake_mon_lunit.assign_coords({'season': seasons})
tlake_sea_lunit = tlake_mon_lunit.groupby('season').mean().chunk({'season':4})

tlake_sea_lunit_inped = xr.apply_ufunc(
    lambda x, y: PchipInterpolator(
        x, y, extrapolate=False)(depths),
    zlake_lunit,#.drop(('lat','lon')),
    tlake_sea_lunit,
    input_core_dims=[['levlak'], ['levlak', 'season', 'ensemble', 'year']],
    output_core_dims=[['depth', 'season', 'ensemble', 'year']],
    dask_gufunc_kwargs={'output_sizes': {'depth': 56}},
    vectorize=True,
    dask='parallelized',
    output_dtypes='float32'
)

tlake_sea_lunit_inped = xr.where(xr.ufuncs.isnan(
    tlake_sea_lunit_inped[:, :, 0, 0, 0]), 0, tlake_sea_lunit_inped)

tlake_pi_std = tlake_sea_lunit_inped.sel(
    year=slice(1850, 1900)).std(('year', 'ensemble'))
tlake_pi_mean = tlake_sea_lunit_inped.sel(
    year=slice(1850, 1900)).mean(('year', 'ensemble'))
tlake_comfort_line = tlake_pi_mean + tlake_pi_std * 2
tlake_mask_window_start = tlake_sea_lunit_inped.mean(
    'ensemble') + 2 * tlake_sea_lunit_inped.std('ensemble') < tlake_comfort_line
tlake_mask_window_end = tlake_sea_lunit_inped.mean(
    'ensemble') - 2 * tlake_sea_lunit_inped.std('ensemble') < tlake_comfort_line

tlake_sig_termin_lunit = xr.apply_ufunc(
    toe,
    tlake_mask_window_end,
    input_core_dims=[['year']],
    vectorize=True,
    dask='parallelized',
    output_dtypes='float32'
).compute()

tlake_sig_termin = np.empty((192, 288, 56, 4))*np.nan
for i in np.arange(4622):
    tlake_sig_termin[lake_jxy[i], lake_ixy[i], :, :
                     ] = tlake_sig_termin_lunit[i,:, :]

tlake_sig_termin = xr.DataArray(
    tlake_sig_termin,
    dims=('lat', 'lon', 'depth', 'season'),
    coords={'depth': depths, 'season':tlake_sig_termin_lunit.season,
            'lat': zlake.lat, 'lon': zlake.lon},
    attrs={'long_name': 'time of emergence for no-analogue condition',
           'units': 'year', 'note': '2101 means no emergence detected'},
    name='TOE_NoAnal'
).transpose('depth', 'season', 'lat', 'lon').reindex(season=['JFM', 'AMJ', 'JAS', 'OND'])

tlake_sig_termin.to_dataset().assign_attrs({'history': 'Calculated by LeiHuang, 20211127'}).to_netcdf(
    '/*your_path/TLAKE_SEA_TOE_NoAnalogue_backward.nc',
    encoding={
        'TOE_NoAnal': {'dtype': 'float32', '_FillValue': 1e+36},
        'lon': {'dtype': 'float32'},
        'lat': {'dtype': 'float32'}
    }
)


# calculate the variability of lake temperature

In [ ]:
tlake_sea_lunit_inped_ensstd = tlake_sea_lunit_inped[:, :, -100:, :].std('ensemble').compute()
tlake_sea_lunit_inped_ensstd_mean = tlake_sea_lunit_inped_ensstd.mean('year')

tlake_varia_sea_mean = np.empty((192, 288, 56, 4))*np.nan
for i in np.arange(4622):
    tlake_varia_sea_mean[lake_jxy[i], lake_ixy[i], :, :
                          ] = tlake_sea_lunit_inped_ensstd_mean[i, :, :].values

tlake_varia_sea_mean = xr.DataArray(
    tlake_varia_sea_mean,
    dims=('lat', 'lon', 'depth', 'season'),
    coords={'depth': depths,
            'lat': zlake.lat, 'lon': zlake.lon, 'season': tlake_sea_lunit_inped_ensstd.season},
    attrs={'long_name': 'climatlogical mean of water temperature variability over 2000-2100',
           'units': 'degree'},
    name='TLAKE_VARIA'
).transpose('depth', 'season', 'lat', 'lon').reindex(season=['JFM', 'AMJ', 'JAS', 'OND'])

tlake_varia_sea_mean.to_dataset().assign_attrs(
    {'history': 'Calculated by LeiHuang, 20211127'}).to_netcdf(
    '/*your_path/TLAKE_SEA_ClimatMeanVariability_2000_2100.nc',
    encoding={
        'TLAKE_VARIA': {'dtype': 'float32', '_FillValue': 1e+36},
        'lon': {'dtype': 'float32'},
        'lat': {'dtype': 'float32'},
        'depth': {'dtype': 'float32'},
    }
)

# calculate the trend of lake temperature

In [2]:
# this function git from https://gist.github.com/luke-gregor/4bb5c483b2d111e52413b260311fbe43
def xarray_trend(xarr,taxis = 'time', units = '\u2103'):    
    from scipy import stats
    #import numexpr as ne
    # getting shapes
    
    m = np.prod(xarr.shape[1:]).squeeze()
    n = xarr.shape[0]
    
    # creating x and y variables for linear regression
    #x = xarr.indexes[taxis].to_pandas().index.to_julian_date().values[:, None]
    #just create a special one only for large ensemble
    x = np.arange(xarr[taxis][0],xarr[taxis][-1:] + 1,1)[:,None]
    y = xarr.to_masked_array().reshape(n, -1)
    
    # ############################ #
    # LINEAR REGRESSION DONE BELOW #
    xm = x.mean(0)  # mean
    ym = y.mean(0)  # mean
    #ya = ne.evaluate("y - ym")
    ya = y - ym  # anomaly
    xa = x - xm  # anomaly
    
    # variance and covariances
    xss = (xa ** 2).sum(0) / (n - 1)  # variance of x (with df as n-1)
    #yss = ne.evaluate("ya ** 2").sum(0) / (n - 1) 
    yss = (ya ** 2).sum(0) / (n - 1)  # variance of y (with df as n-1)
    #xys = ne.evaluate("xa * ya").sum(0) / (n - 1)
    xys = (xa * ya).sum(0) / (n - 1)  # covariance (with df as n-1)
    # slope and intercept
    slope = xys / xss
    intercept = ym - (slope * xm)
    # statistics about fit
    df = n - 2
    r = xys / (xss * yss)**0.5
    t = r * (df / ((1 - r) * (1 + r)))**0.5
    p = stats.distributions.t.sf(abs(t), df)
    
    # misclaneous additional functions
    # yhat = dot(x, slope[None]) + intercept
    # sse = ((yhat - y)**2).sum(0) / (n - 2)  # n-2 is df
    # se = ((1 - r**2) * yss / xss / df)**0.5
    
    # preparing outputs
    out = xarr[:2].mean(taxis)
    # first create variable for slope and adjust meta
    xarr_slope = out.copy()
    xarr_slope.name = '_slope'
    xarr_slope.attrs['units'] = units + ' / ' + taxis
    xarr_slope.values = slope.reshape(xarr.shape[1:])
    # do the same for the p value
    xarr_p = out.copy()
    xarr_p.name = '_Pvalue'
    xarr_p.attrs['info'] = "If p < 0.05 then the results from 'slope' are significant."
    xarr_p.values = p.reshape(xarr.shape[1:])
    # join these variables
    xarr_out = xarr_slope.to_dataset(name='slope')
    xarr_out['pval'] = xarr_p

    return xarr_out

In [ ]:
tlake_sea_lunit_inped_ensmean = tlake_sea_lunit_inped[:, :, -100:, :].mean('ensemble').compute()
tlake_sea_lunit_inped_ensmean_trend = xarray_trend(
    tlake_sea_lunit_inped_ensmean.transpose('year',...), taxis = 'year').slope

tlake_sea_trend = np.empty((192, 288, 56, 4))*np.nan
for i in np.arange(4622):
    tlake_sea_trend[lake_jxy[i], lake_ixy[i], :, :
                    ] = tlake_sea_lunit_inped_ensmean_trend[i, :, :].values

tlake_sea_trend = xr.DataArray(
    tlake_sea_trend,
    dims=('lat', 'lon', 'depth', 'season'),
    coords={'depth': depths,
            'lat': zlake.lat, 'lon': zlake.lon, 'season': tlake_sea_lunit_inped_ensmean.season},
    attrs={'long_name': 'linear trend of water temperature over 2000-2100',
           'units': 'degree/year'},
    name='linear_trend'
).transpose('depth', 'season', 'lat', 'lon').reindex(season=['JFM', 'AMJ', 'JAS', 'OND'])

tlake_sea_trend.to_dataset().assign_attrs(
    {'history': 'Calculated by LeiHuang, 20211127'}).to_netcdf(
    '/*your_path/TLAKE_SEA_Trend_2000_2100.nc',
    encoding={
        'linear_trend': {'dtype': 'float32', '_FillValue': 1e+36},
        'lon': {'dtype': 'float32'},
        'lat': {'dtype': 'float32'},
        'depth': {'dtype': 'float32'},
    }
)